In [1]:
import os 

import pandas as pd
import numpy as np

# from pathlib import Path


In [2]:
directory ='/DataVelocity_20210208'

In [3]:
cond1 = 'ctl_apoptoticNuclei'
cond2 = 'drug_apoptoticNuclei'

## Cond 1 = apoptoticnuclei

In [4]:
# Cond1 
working_directory = os.path.join(directory, cond1)
# List file in the working directory
list_file = os.listdir(working_directory)


In [5]:
# Combine initial information into 3 tables according to each time interval acquisition
df_60 = pd.DataFrame(np.nan*np.zeros((20,1)))
df_80 = pd.DataFrame(np.nan*np.zeros((20,1)))
df_90 = pd.DataFrame(np.nan*np.zeros((20,1)))

for f in list_file :
    df = pd.read_csv(os.path.join(working_directory, f), sep=';', encoding = "latin")
    df.columns = df.columns.str.lower()
    if df.loc[0]['time(s)']==60:
        # remove line with NaN value = control information 
        df.dropna(inplace = True)
        df_60[f.split('_')[0]] = df['norm veloc']
        
    if df.loc[0]['time(s)']==80:
        # remove line with NaN value = control information 
        df.dropna(inplace = True)
        df_80[f.split('_')[0]] = df['norm veloc']
        
    if df.loc[0]['time(s)']==90:
        # remove line with NaN value = control information 
        df.dropna(inplace = True)
        df_90[f.split('_')[0]] = df['norm veloc']    

        
# Remove the first column, which has be used to avoid trunkat data
df_60.drop(0, axis=1, inplace=True)
# Remove line which contains no information
df_60.dropna(axis=0, how='all', inplace=True)
# Transpose matrix
df_60 = df_60.T
# Rename column according to time interval
nb_t = df_60.shape[1]
columns = np.arange(0, 60*nb_t, 60)
df_60.columns = columns

# Remove the first column, which has be used to avoid trunkat data
df_80.drop(0, axis=1, inplace=True)
# Remove line which contains no information
df_80.dropna(axis=0, how='all', inplace=True)
# Transpose matrix
df_80 = df_80.T
# Rename column according to time interval
nb_t = df_80.shape[1]
columns = np.arange(0, 80*nb_t, 80)
df_80.columns = columns

# Remove the first column, which has be used to avoid trunkat data
df_90.drop(0, axis=1, inplace=True)
# Remove line which contains no information
df_90.dropna(axis=0, how='all', inplace=True)
# Transpose matrix
df_90 = df_90.T
# Rename column according to time interval
nb_t = df_90.shape[1]
columns = np.arange(0, 90*nb_t, 90)
df_90.columns = columns

In [6]:
# If there is information in the dataframe
# Extrapolate velocity
try:
    df_60 = df_60.T
    initial_time = df_60.index.to_list()
    for i in range(0, df_60.index.max()+10,  10):
        if i not in df_60.index : 
            df_60.loc[i] = df_60.loc[i//60*60]+(df_60.loc[(i//60+1)*60]-df_60.loc[i//60*60])/60*(i%60)

    df_60.sort_index(inplace=True)
    df_60.to_csv(directory+'/df_60_ctl_apoptotic.csv')

# otherwise, do nothing
except:
    pass

In [7]:
# If there is information in the dataframe
# Extrapolate velocity
try:
    df_80 = df_80.T
    initial_time = df_80.index.to_list()
    for i in range(0, df_80.index.max()+10,  10):
        if i not in df_80.index : 
            df_80.loc[i] = df_80.loc[i//80*80]+(df_80.loc[(i//80+1)*80]-df_80.loc[i//80*80])/80*(i%80)
    df_80.sort_index(inplace=True)
    df_60.to_csv(directory+'/df_80_ctl_apoptotic.csv')

# otherwise, do nothing
except:
    pass

In [8]:
# If there is information in the dataframe
# Extrapolate velocity
try :
    df_90 = df_90.T
    initial_time = df_90.index.to_list()
    for i in range(0, df_90.index.max()+10,  10):
        if i not in df_90.index : 
            df_90.loc[i] = df_90.loc[i//90*90]+(df_90.loc[(i//90+1)*90]-df_90.loc[i//90*90])/90*(i%90)
            
    df_90.sort_index(inplace=True)
    df_90.to_csv(directory+'/df_90_ctl_apoptotic.csv')
    
# otherwise, do nothing
except:
    pass

In [9]:
# Combine the three tables into one
df_all = df_60.copy()
df_all = pd.concat((df_all, df_80), axis=1)
df_all = pd.concat((df_all, df_90), axis=1)
df_all.to_csv(directory+'/df_all_ctl_apoptotic.csv')

In [10]:
# Align on peak
before_peak_align = pd.DataFrame(np.zeros((df_all.shape[0],1)))
after_peak_align = pd.DataFrame(np.zeros((df_all.shape[0],1)))

for col_name, values in df_all.iteritems():
    before_peak_align[col_name] = pd.DataFrame(values[:np.argmax(values)][::-1].to_numpy())
    after_peak_align[col_name] = pd.DataFrame(values[np.argmax(values):].to_numpy())
    
before_peak_align.drop(0, axis=1, inplace=True)
before_peak_align.dropna(axis=0, how='all', inplace=True)
after_peak_align.drop(0, axis=1, inplace=True)
after_peak_align.dropna(axis=0, how='all', inplace=True)

before_peak_align = before_peak_align[::-1]
before_peak_align.reset_index(inplace=True, drop=True)

/home/admin-suz/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [11]:
align_result = pd.concat((before_peak_align, after_peak_align), axis=0)
align_result.reset_index(inplace=True, drop=True)
align_result.to_csv(directory+'/df_all_ctl_apoptotic_align.csv')

## Cond 2 : 'non-apoptoticNuclei/xls'

In [12]:
# Cond2
working_directory = os.path.join(directory, cond2)
list_file = os.listdir(working_directory)

In [13]:
# Combine initial information into 3 tables according to each time interval acquisition
df_60 = pd.DataFrame(np.nan*np.zeros((20,1)))
df_80 = pd.DataFrame(np.nan*np.zeros((20,1)))
df_90 = pd.DataFrame(np.nan*np.zeros((20,1)))

for f in list_file :
    print(f)
    df = pd.read_csv(os.path.join(working_directory, f), sep=';', encoding = "latin")
    df.columns = df.columns.str.lower()
    
    if df.loc[0]['time (sec)']==60:
        # remove line with NaN value = control information 
        df.dropna(inplace = True)
        df_60[f.split('_')[0]] = df['norm veloc']
    
    if df.loc[0]['time (sec)']==80:
        # remove line with NaN value = control information 
        df.dropna(inplace = True)
        df_80[f.split('_')[0]] = df['norm veloc']
        
    if df.loc[0]['time (sec)']==90:
        # remove line with NaN value = control information 
        df.dropna(inplace = True)
        df_90[f.split('_')[0]] = df['norm veloc']
        
df_60.drop(0, axis=1, inplace=True)
df_60.dropna(axis=0, how='all', inplace=True)
df_60 = df_60.T
nb_t = df_60.shape[1]
columns = np.arange(0, 60*nb_t, 60)
df_60.columns = columns

df_80.drop(0, axis=1, inplace=True)
df_80.dropna(axis=0, how='all', inplace=True)
df_80 = df_80.T
nb_t = df_80.shape[1]
columns = np.arange(0, 80*nb_t, 80)
df_80.columns = columns

df_90.drop(0, axis=1, inplace=True)
df_90.dropna(axis=0, how='all', inplace=True)
df_90 = df_90.T
nb_t = df_90.shape[1]
columns = np.arange(0, 90*nb_t, 90)
df_90.columns = columns

dnRho1_20201013-Gc3ai_nlsscarlet_dnRHO2ug_FaiSc_e2_5ss_driftcorr-middle_nt1_combi.avi_tr.csv
apo3_Y_20201002-Gc3ai_nlsscar_Y27632-100um_FaiSc_e2_part2_5ss_bot_nt1.csv
apo10_Y_20201016-Gc3ai_nlsscarlet_20uMY27632_FaiSc_e2_part1_6ss_driftcorr-bottom_nt1_tr.csv
apo11_Y_20201211-Gc3ai_nlsscarlet_FAiSc_Y27632_100uM_e2_driftcorr-bottom_nt1_tr.csv
apo1_Y_20201002-Gc3ai_nlsscarlet_Y27632-100um_FaiSc_e1_part2_5ss_driftcorr-top_rot-16_nt1.csv
dnRho3_20210126-GC3a-nlsScarlet_dnRho2ug_FAiSc_e2_part2_5ss_driftcorr-top_nt1_combi.avi_tr.csv
dnRho2_20210126-GC3a-nlsScarlet_dnRho2ug_FAiSc_e1_part2_5ss_driftcorr-middle_nt2_combi.avi_tr.csv
apo4_Y_20201009-Gc3ai_nlsscarlet_Y27632-100um_FaiSc_e1_6ss_nt1_tr.csv
dnRho4_20210126-GC3a-nlsScarlet_dnRho2ug_FAiSc_e2_part2_5ss_driftcorr-top_nt2_combi.avi_tr.csv


In [14]:
try :
    df_60 = df_60.T
    initial_time = df_60.index.to_list()
    for i in range(0, df_60.index.max()+10,  10):
        if i not in df_60.index : 
            df_60.loc[i] = df_60.loc[i//60*60]+(df_60.loc[(i//60+1)*60]-df_60.loc[i//60*60])/60*(i%60)

    df_60.sort_index(inplace=True)
    df_60.to_csv(directory+'/df_60_drug_apoptotic.csv')
except:
    pass

In [15]:
try:
    df_80 = df_80.T
    initial_time = df_80.index.to_list()
    for i in range(0, df_80.index.max()+10,  10):
        if i not in df_80.index : 
            df_80.loc[i] = df_80.loc[i//80*80]+(df_80.loc[(i//80+1)*80]-df_80.loc[i//80*80])/80*(i%80)
            
    df_80.sort_index(inplace=True)
    df_80.to_csv(directory+'/df_80_drug_apoptotic.csv')
except:
    pass

In [16]:
try:
    df_90 = df_90.T
    initial_time = df_90.index.to_list()
    for i in range(0, df_90.index.max()+10,  10):
        if i not in df_90.index : 
            df_90.loc[i] = df_90.loc[i//90*90]+(df_90.loc[(i//90+1)*90]-df_90.loc[i//90*90])/90*(i%90)
            
    df_90.sort_index(inplace=True)
    df_90.to_csv(directory+'/df_90_drug_apoptotic.csv')
except:
    pass

In [17]:
df_all = df_60.copy()
df_all = pd.concat((df_all, df_80), axis=1)
df_all = pd.concat((df_all, df_90), axis=1)
df_all.to_csv(directory+'/df_all_drug_apoptotic.csv')

In [18]:
# Align on peak
before_peak_align = pd.DataFrame(np.zeros((df_all.shape[0],1)))
after_peak_align = pd.DataFrame(np.zeros((df_all.shape[0],1)))

for col_name, values in df_all.iteritems():
    before_peak_align[col_name] = pd.DataFrame(values[:np.argmax(values)][::-1].to_numpy())
    after_peak_align[col_name] = pd.DataFrame(values[np.argmax(values):].to_numpy())
    
before_peak_align.drop(0, axis=1, inplace=True)
before_peak_align.dropna(axis=0, how='all', inplace=True)
after_peak_align.drop(0, axis=1, inplace=True)
after_peak_align.dropna(axis=0, how='all', inplace=True)

before_peak_align = before_peak_align[::-1]
before_peak_align.reset_index(inplace=True, drop=True)

In [19]:
align_result = pd.concat((before_peak_align, after_peak_align), axis=0)
align_result.reset_index(inplace=True, drop=True)
align_result.to_csv(directory+'/df_all_drug_apoptotic_align.csv')